# Pyspark_Association_Rdd_Grocery

In [1]:
#1 - import module
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.mllib.fpm import FPGrowth
from itertools import combinations

In [2]:
#2 - Create SparkContext
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.history.fs.cleaner.interval', u'7d'),
 (u'spark.shuffle.io.serverThreads', u'128'),
 (u'spark.yarn.historyServer.address',
  u'hdp-all-spark-new.c.kao-project-v1.internal:18081'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.driver.host', u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.sql.statistics.fallBackToHdfs', u'true'),
 (u'spark.shuffle.file.buffer', u'1m'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark

In [5]:
print sc.getConf().toDebugString() 

spark.app.id=application_1541652388947_0030
spark.app.name=pyspark-shell
spark.driver.appUIAddress=http://hdp-all-spark-new.c.kao-project-v1.internal:4040
spark.driver.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.driver.host=hdp-all-spark-new.c.kao-project-v1.internal
spark.driver.port=34761
spark.eventLog.dir=hdfs:///spark2-history/
spark.eventLog.enabled=true
spark.executor.extraJavaOptions=-XX:+UseNUMA
spark.executor.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.executor.id=driver
spark.executorEnv.PYTHONPATH=/usr/hdp/3.0.0.0-1634/spark2/python:/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip
spark.history.fs.cleaner.enabled=true
spark.history.fs.cleaner.interval=7d
spark.history.fs.cleaner.maxAge=90d
spark.history.fs.logDirectory=hdfs:///spark2-history/
spark.histo

In [6]:
#3 - Read file to spark RDD

data = sc.textFile("/user/admin/data/grocery.csv")
data.cache()

/user/admin/data/grocery.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
#4 - Print sample 5 rows of all variables
for line in data.top(5):
    print line 

TransactionID,Product
33707,vinegar
33707,tropical fruit
33707,shopping bags
33707,other vegetables


In [8]:
# Print sample 5 rows of target variable
for line in data.top(5):
    print (line.split(','))[-1]

Product
vinegar
tropical fruit
shopping bags
other vegetables


In [9]:
# List all features
header = data.first()
header_list = header.split(',')

print "number of features : " + str(len(header_list) - 1)
print header

number of features : 1
TransactionID,Product


In [10]:
#label stat
print "\nlabel statistic"
label_stat = sorted(data.map(lambda x : x.split(',')[-1])
              .map(lambda label : (label,1))
              .countByKey()
              .items())
for line in label_stat:
    print line


label statistic
(u'Instant food products', 79)
(u'Product', 1)
(u'UHT-milk', 329)
(u'abrasive cleaner', 35)
(u'artif. sweetener', 32)
(u'baby cosmetics', 6)
(u'baby food', 1)
(u'bags', 4)
(u'baking powder', 174)
(u'bathroom cleaner', 27)
(u'beef', 516)
(u'berries', 327)
(u'beverages', 256)
(u'bottled beer', 792)
(u'bottled water', 1087)
(u'brandy', 41)
(u'brown bread', 638)
(u'butter', 545)
(u'butter milk', 275)
(u'cake bar', 130)
(u'candles', 88)
(u'candy', 294)
(u'canned beer', 764)
(u'canned fish', 148)
(u'canned fruit', 32)
(u'canned vegetables', 106)
(u'cat food', 229)
(u'cereals', 56)
(u'chewing gum', 207)
(u'chicken', 422)
(u'chocolate', 488)
(u'chocolate marshmallow', 89)
(u'citrus fruit', 814)
(u'cleaner', 50)
(u'cling film/bags', 112)
(u'cocoa drinks', 22)
(u'coffee', 571)
(u'condensed milk', 101)
(u'cooking chocolate', 25)
(u'cookware', 27)
(u'cream', 13)
(u'cream cheese ', 390)
(u'curd', 524)
(u'curd cheese', 50)
(u'decalcifier', 15)
(u'dental care', 57)
(u'dessert', 365)


In [11]:
#5 - Remove header from data
data = data.filter(lambda x: x!=header)
for line in data.top(5):
    print line

33707,vinegar
33707,tropical fruit
33707,shopping bags
33707,other vegetables
33707,chicken


In [12]:
#6 - Transform data to key-value
data = data.map(lambda x : tuple(x.split(',')))
for line in data.top(5):
    print line

(u'33707', u'vinegar')
(u'33707', u'tropical fruit')
(u'33707', u'shopping bags')
(u'33707', u'other vegetables')
(u'33707', u'chicken')


In [13]:
#7 - Group item which has same key(transaction)
data = data.groupByKey().mapValues(list)
for line in data.top(5):
    print line

(u'33707', [u'chicken', u'tropical fruit', u'other vegetables', u'vinegar', u'shopping bags'])
(u'33706', [u'semi-finished bread', u'bottled water', u'soda', u'bottled beer'])
(u'33705', [u'chicken', u'citrus fruit', u'other vegetables', u'butter', u'yogurt', u'frozen dessert', u'domestic eggs', u'rolls/buns', u'rum', u'cling film/bags'])
(u'33704', [u'cooking chocolate'])
(u'33703', [u'sausage', u'chicken', u'beef', u'hamburger meat', u'citrus fruit', u'grapes', u'root vegetables', u'whole milk', u'butter', u'whipped/sour cream', u'flour', u'coffee', u'red/blush wine', u'salty snack', u'chocolate', u'hygiene articles', u'napkins'])


In [14]:
#8 - Select only items
data = data.map(lambda (transactionId,items) : items)
for line in data.top(5):
    print line

[u'zwieback', u'salty snack', u'popcorn', u'waffles', u'chocolate']
[u'zwieback', u'oil', u'mustard', u'chocolate']
[u'zwieback', u'long life bakery product', u'dental care']
[u'zwieback', u'long life bakery product']
[u'zwieback', u'dishes']


In [15]:
#9 - Train model
model = FPGrowth.train(data, minSupport=0.02, numPartitions=3)

print model

In [16]:
#10 - Print result from FPGrowth model
item_sets = model.freqItemsets().collect()
for item_set in item_sets:
    print item_set

FreqItemset(items=[u'frozen meals'], freq=279)
FreqItemset(items=[u'hard cheese'], freq=241)
FreqItemset(items=[u'dessert'], freq=365)
FreqItemset(items=[u'napkins'], freq=515)
FreqItemset(items=[u'brown bread'], freq=638)
FreqItemset(items=[u'brown bread', u'whole milk'], freq=248)
FreqItemset(items=[u'sausage'], freq=924)
FreqItemset(items=[u'sausage', u'rolls/buns'], freq=301)
FreqItemset(items=[u'sausage', u'other vegetables'], freq=265)
FreqItemset(items=[u'sausage', u'soda'], freq=239)
FreqItemset(items=[u'sausage', u'whole milk'], freq=294)
FreqItemset(items=[u'butter milk'], freq=275)
FreqItemset(items=[u'margarine'], freq=576)
FreqItemset(items=[u'margarine', u'whole milk'], freq=238)
FreqItemset(items=[u'grapes'], freq=220)
FreqItemset(items=[u'soda'], freq=1715)
FreqItemset(items=[u'soda', u'rolls/buns'], freq=377)
FreqItemset(items=[u'soda', u'other vegetables'], freq=322)
FreqItemset(items=[u'soda', u'whole milk'], freq=394)
FreqItemset(items=[u'bottled beer'], freq=792)
F

In [17]:
#11 - Find support list
num_transactions = float(data.count())
print "number of transactions: "+ str(num_transactions)

item_sets_with_support = {}
for item_set in item_sets:
    item_sets_with_support[frozenset(item_set.items)] = item_set.freq/num_transactions
    
for support in item_sets_with_support.iteritems():
    print support

number of transactions: 9835.0
(frozenset([u'yogurt', u'whipped/sour cream']), 0.02074224707676665)
(frozenset([u'pork', u'other vegetables']), 0.021657346212506354)
(frozenset([u'other vegetables', u'root vegetables']), 0.047381799694966954)
(frozenset([u'beef']), 0.05246568378240976)
(frozenset([u'whole milk', u'whipped/sour cream']), 0.032231825114387394)
(frozenset([u'whole milk', u'frozen vegetables']), 0.02043721403152008)
(frozenset([u'rolls/buns']), 0.18393492628368074)
(frozenset([u'yogurt', u'root vegetables']), 0.025826131164209457)
(frozenset([u'rolls/buns', u'soda']), 0.03833248601931876)
(frozenset([u'tropical fruit', u'whole milk']), 0.04229791560752415)
(frozenset([u'domestic eggs']), 0.06344687341128623)
(frozenset([u'whole milk', u'bottled beer']), 0.02043721403152008)
(frozenset([u'rolls/buns', u'pastry']), 0.02094560244026436)
(frozenset([u'whole milk', u'curd']), 0.026131164209456024)
(frozenset([u'white bread']), 0.042094560244026434)
(frozenset([u'pork']), 0.0576

In [18]:
#12 - Find rules and calculate confident and lift
rules = []
for (item_set,item_set_supp) in item_sets_with_support.iteritems():
    if(len(item_set)>1):
        for (LHS,LHS_supp) in item_sets_with_support.iteritems():
            if(LHS < item_set):
                RHS = item_set - LHS
                RHS_supp = item_sets_with_support[RHS]
                confidence = item_set_supp/LHS_supp
                lift = item_set_supp/(LHS_supp*RHS_supp)
                rules.append([LHS,RHS,item_set_supp,confidence,lift])

rules = sorted(rules, key=lambda x: x[3], reverse=True)

In [19]:
#13 - Convert list to Dataframe 
# [LHS,RHS,item_set_supp,confidence,lift] = rule[0,1,2,3,4]
tuple_rules = [tuple([list(rule[0]),list(rule[1]),rule[2],rule[3],rule[4]]) for rule in rules]
rule_rdd = sc.parallelize(tuple_rules)
rule_rdd.top(1)

[([u'yogurt', u'whole milk'],
  [u'other vegetables'],
  0.02226741230299949,
  0.39745916515426494,
  2.0541307878571704)]

In [20]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .appName("Pyspark_Association_Rdd_Grocery")
         .getOrCreate())

In [21]:
rule_df = rule_rdd.toDF(["LHS","RHS","sup","conf","lift"])
rule_df.sort("conf",ascending=False).toPandas()

LHS                                  RHS  \
0             [yogurt, other vegetables]                         [whole milk]   
1                               [butter]                         [whole milk]   
2                                 [curd]                         [whole milk]   
3    [other vegetables, root vegetables]                         [whole milk]   
4          [whole milk, root vegetables]                   [other vegetables]   
5                        [domestic eggs]                         [whole milk]   
6                   [whipped/sour cream]                         [whole milk]   
7                      [root vegetables]                         [whole milk]   
8                      [root vegetables]                   [other vegetables]   
9                    [frozen vegetables]                         [whole milk]   
10                           [margarine]                         [whole milk]   
11                                [beef]                         [whole milk]   
12                      [tropical fruit]                         [whole milk]   
13                  [whipped/sour cream]                   [other vegetables]   
14                              [yogurt]                         [whole milk]   
15                           [pip fruit]                         [whole milk]   
16                  [yogurt, whole milk]                   [other vegetables]   
17                         [brown bread]                         [whole milk]   
18                    [other vegetables]                         [whole milk]   
19                                [pork]                         [whole milk]   
20                                [pork]                   [other vegetables]   
21                              [pastry]                         [whole milk]   
22                        [citrus fruit]                         [whole milk]   
23               [fruit/vegetable juice]                         [whole milk]   
24                              [butter]                   [other vegetables]   
25                       [domestic eggs]                   [other vegetables]   
26                        [citrus fruit]                   [other vegetables]   
27                         [frankfurter]                         [whole milk]   
28                           [pip fruit]                   [other vegetables]   
29                          [newspapers]                         [whole milk]   
..                                   ...                                  ...   
104                         [whole milk]                 [whipped/sour cream]   
105                               [soda]                             [pastry]   
106                   [other vegetables]                      [shopping bags]   
107                   [other vegetables]        [whole milk, root vegetables]   
108                               [soda]                     [tropical fruit]   
109                         [whole milk]                       [citrus fruit]   
110                         [whole milk]                          [pip fruit]   
111                         [whole milk]                      [domestic eggs]   
112                         [whole milk]                            [sausage]   
113                   [other vegetables]                             [pastry]   
114                   [other vegetables]                      [domestic eggs]   
115                   [other vegetables]                 [yogurt, whole milk]   
116                         [rolls/buns]                             [pastry]   
117                   [other vegetables]                               [pork]   
118                   [other vegetables]              [fruit/vegetable juice]   
119                         [whole milk]                             [butter]   
120                         [whole milk]                         [newspapers]   
121                         [whole milk]              [fruit/vegetab

In [22]:
#14 - Show result sort by desc(conf) and desc(lift)
rule_df.sort("conf",ascending=False).filter(rule_df["conf"] > 0.4).toPandas()

LHS                 RHS       sup  \
0            [yogurt, other vegetables]        [whole milk]  0.022267   
1                              [butter]        [whole milk]  0.027555   
2                                [curd]        [whole milk]  0.026131   
3   [other vegetables, root vegetables]        [whole milk]  0.023183   
4         [whole milk, root vegetables]  [other vegetables]  0.023183   
5                       [domestic eggs]        [whole milk]  0.029995   
6                  [whipped/sour cream]        [whole milk]  0.032232   
7                     [root vegetables]        [whole milk]  0.048907   
8                     [root vegetables]  [other vegetables]  0.047382   
9                   [frozen vegetables]        [whole milk]  0.020437   
10                          [margarine]        [whole milk]  0.024199   
11                               [beef]        [whole milk]  0.021251   
12                     [tropical fruit]        [whole milk]  0.042298   
13                 [whipped/sour cream]  [other vegetables]  0.028876   
14                             [yogurt]        [whole milk]  0.056024   

        conf      lift  
0   0.512881  2.007235  
1   0.497248  1.946053  
2   0.490458  1.919481  
3   0.489270  1.914833  
4   0.474012  2.449770  
5   0.472756  1.850203  
6   0.449645  1.759754  
7   0.448694  1.756031  
8   0.434701  2.246605  
9   0.424947  1.663094  
10  0.413194  1.617098  
11  0.405039  1.585180  
12  0.403101  1.577595  
13  0.402837  2.081924  
14  0.401603  1.571735

In [23]:
rule_df.sort("lift",ascending=False).filter(rule_df["conf"] > 0.4).toPandas()

LHS                 RHS       sup  \
0         [whole milk, root vegetables]  [other vegetables]  0.023183   
1                     [root vegetables]  [other vegetables]  0.047382   
2                  [whipped/sour cream]  [other vegetables]  0.028876   
3            [yogurt, other vegetables]        [whole milk]  0.022267   
4                              [butter]        [whole milk]  0.027555   
5                                [curd]        [whole milk]  0.026131   
6   [other vegetables, root vegetables]        [whole milk]  0.023183   
7                       [domestic eggs]        [whole milk]  0.029995   
8                  [whipped/sour cream]        [whole milk]  0.032232   
9                     [root vegetables]        [whole milk]  0.048907   
10                  [frozen vegetables]        [whole milk]  0.020437   
11                          [margarine]        [whole milk]  0.024199   
12                               [beef]        [whole milk]  0.021251   
13                     [tropical fruit]        [whole milk]  0.042298   
14                             [yogurt]        [whole milk]  0.056024   

        conf      lift  
0   0.474012  2.449770  
1   0.434701  2.246605  
2   0.402837  2.081924  
3   0.512881  2.007235  
4   0.497248  1.946053  
5   0.490458  1.919481  
6   0.489270  1.914833  
7   0.472756  1.850203  
8   0.449645  1.759754  
9   0.448694  1.756031  
10  0.424947  1.663094  
11  0.413194  1.617098  
12  0.405039  1.585180  
13  0.403101  1.577595  
14  0.401603  1.571735